In [ ]:
import numpy as np
import networkx as nx
import tensorflow as tf
import seaborn as sns
import pandas as pd
sns.set()

In [ ]:
def grid_adjacency(N):
  A = np.zeros((N**2,N**2))
  for i in range(0,N):
    for j in range(0,N):
      if j!=N-1:
        A[N*i+j,N*i+j+1]=1
        A[N*i+j+1,N*i+j]=1
      if i!=N-1:
        A[N*i+j,N*(i+1)+j]=1
        A[N*(i+1)+j,N*i+j]=1
  return A
def gnn(fts,adj,W,activation):
  seq_fts = tf.matmul(W,fts)
  ret_fts = tf.matmul(adj,seq_fts)
  return activation(ret_fts)
def DirichletEnergy(fts,adj):
  [N,F] = np.shape(fts)
  E = 0
  for i in range(0,N):
    for j in range(0,N):
      if adj[i,j]==1:
        E+=np.linalg.norm(fts[i,:]-fts[j,:])**2
  return (1/N)*E


In [ ]:
fts = np.random.rand(100,5)
Y = np.random.rand(100,5)
adj = grid_adjacency(10)
adj_with_self_loop = adj+np.eye(100)
deg = tf.reduce_sum(adj_with_self_loop,axis=-1)
norm_deg = tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg,tf.matmul(adj_with_self_loop,norm_deg))
DirichletEnergyCON = np.zeros((100,1))
for layers in range(100):
  W = 0.01*np.random.rand(100,100)
  fts = gnn(fts,norm_adj,W,tf.nn.relu)
  DirichletEnergyCON[layers] = DirichletEnergy(fts,adj)
DirichletEnergyCON = np.concatenate( DirichletEnergyCON, axis=0 )

In [ ]:
def gnn_kuramoto(theta,adj,k,omega,dt,activation,W,gnn_fn):
  [N,F] = np.shape(theta)
  theta = theta + dt*(omega*np.ones((N,F)) + (k/N)*tf.matmul(adj,np.sin(gnn_fn(theta,adj,W,activation))))
  return theta

In [ ]:
fts = np.random.rand(100,5)
DirichletEnergyKuramoto = np.zeros((100,1))
for layers in range(100):
  W = 0.01*np.random.rand(100,100)
  fts = gnn_kuramoto(fts,norm_adj,1,1,0.1,tf.nn.relu,W,gnn)
  DirichletEnergyKuramoto[layers] = DirichletEnergy(fts,adj)
DirichletEnergyKuramoto = np.concatenate( DirichletEnergyKuramoto, axis=0 )

In [ ]:
def gnn_hopf_rk4(X,Y,adj,a,beta,omega,k,dt,activation,transform,gnn_fn):
  [N,F] = np.shape(X)
  xk1 = a*X + (np.power(X,2) + np.power(Y,2))*(beta*Y-X) - omega*Y + k*activation(gnn_fn(X,adj,transform,activation))
  yk1 = a*Y - (np.power(X,2) + np.power(Y,2))*(beta*Y+X) - omega*X + k*activation(gnn_fn(Y,adj,transform,activation))

  xk2 = a*(X+dt*0.5*xk1) + (np.power(X+dt*0.5*xk1,2) + np.power(Y+dt*0.5*yk1,2))*(beta*(Y+dt*0.5*yk1)-(X+dt*0.5*xk1)) - omega*(Y+dt*0.5*yk1) + k*activation(gnn_fn(X+dt*0.5*xk1,adj,transform,activation))
  yk2 = a*(Y+dt*0.5*yk1) - (np.power(X+dt*0.5*xk1,2) + np.power(Y+dt*0.5*yk1,2))*(beta*(Y+dt*0.5*yk1)+(X+dt*0.5*xk1)) - omega*(X+dt*0.5*xk1) + k*activation(gnn_fn(Y+dt*0.5*yk1,adj,transform,activation))

  xk3 = a*(X+dt*0.5*xk2) + (np.power(X+dt*0.5*xk2,2) + np.power(Y+dt*0.5*yk2,2))*(beta*(Y+dt*0.5*yk2)-(X+dt*0.5*xk2)) - omega*(Y+dt*0.5*yk2) + k*activation(gnn_fn(X+dt*0.5*xk2,adj,transform,activation))
  yk3 = a*(Y+dt*0.5*yk2) - (np.power(X+dt*0.5*xk2,2) + np.power(Y+dt*0.5*yk2,2))*(beta*(Y+dt*0.5*yk2)+(X+dt*0.5*xk2)) - omega*(X+dt*0.5*xk2) + k*activation(gnn_fn(Y+dt*0.5*yk2,adj,transform,activation))
  
  xk4 = a*(X+dt*xk3) + (np.power(X+dt*xk3,2) + np.power(Y+dt*yk3,2))*(beta*(Y+dt*yk3)-(X+dt*xk3)) - omega*(Y+dt*yk3) + k*activation(gnn_fn(X+dt*xk3,adj,transform,activation))
  yk4 = a*(Y+dt*yk3) - (np.power(X+dt*xk3,2) + np.power(Y+dt*yk3,2))*(beta*(Y+dt*yk3)+(X+dt*xk3)) - omega*(X+dt*xk3) + k*activation(gnn_fn(Y+dt*yk3,adj,transform,activation))
  
  X = X + (dt/6)*(xk1+2*xk2+2*xk3+xk4)
  Y = Y + (dt/6)*(yk1+2*yk2+2*yk3+yk4)
  return [X,Y]

In [ ]:
fts = np.random.rand(100,5)
Y = np.random.rand(100,5)

DirichletEnergyHopf = np.zeros((100,1))
for layers in range(100):
  W = 0.01*np.random.rand(100,100)
  [fts,Y] = gnn_hopf_rk4(fts,Y,norm_adj,-0.02,0,2,1,0.01,tf.nn.relu,W,gnn)
  DirichletEnergyHopf[layers] = DirichletEnergy(fts,adj)
DirichletEnergyHopf = np.concatenate( DirichletEnergyHopf, axis=0 )

In [ ]:
def gnn_xy(theta,p,adj,J,k_avg,dt,activation,W,gnn_fn):
  [N,F] = np.shape(theta)
  p = p + dt*(-J/k_avg)*tf.matmul(adj,np.sin(gnn_fn(theta,adj,W,activation)))
  theta = theta + dt*p
  return [theta,p]

In [ ]:
T=0.2;
fts = np.random.rand(100,5)
p = np.random.normal(0,T,(100,5))
k_avg = sum(sum(adj))/100;

DirichletEnergyXY = np.zeros((100,1))
for layers in range(100):
  W = 0.01*np.random.rand(100,100)
  [fts,p] = gnn_xy(fts,p,norm_adj,1,k_avg,0.01,tf.nn.relu,W,gnn)
  DirichletEnergyXY[layers] = DirichletEnergy(fts,adj)
DirichletEnergyXY = np.concatenate( DirichletEnergyXY, axis=0 )

In [ ]:
df = pd.DataFrame({'GCN': DirichletEnergyCON, 'Kuramoto-GCN': DirichletEnergyKuramoto, 'Hopf-GCN': DirichletEnergyHopf, 'XY-GCN' : DirichletEnergyXY})
g = sns.lineplot(data=df)
g.set_yscale("log")
g.set_xscale("log")
g.set_xlabel('Layer')
g.set_ylabel('Dirichlet energy')